## BERT embeddings


In [4]:
import tensorflow_hub as hub

In [5]:
!pip install tensorflow_text
import tensorflow_text as text  # Registers the ops.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.0 MB/s eta 0:00:00


In [ ]:
preprocess_url = "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3"
encoder_url = "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2"

In [6]:
import tensorflow as tf

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].


In [7]:
my_text = ["Rohit Gurunath Sharma (born 30 April 1987) is an Indian international cricketer who currently captains the India national cricket team across all formats", "Considered one of the best batsmen of his generation and one of the greatest opening batters of all time","[3] Sharma is known for his timing, elegance, six-hitting abilities and leadership skills.", "Sharma holds several batting records which famously includes most double centuries in ODI cricket (3),most centuries at Cricket World Cups (7) and most hundreds in Twenty20 Internationals(5).","He is a right-handed batsman. He plays for Mumbai Indians in IPL and for Mumbai in domestic cricket."]

In [8]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(my_text)
bert_results = embedding_model(sentences)
bert_results

<tf.Tensor: shape=(5, 128), dtype=float32, numpy=
array([[-2.12475047e-01,  6.90542519e-01, -1.17348582e-01,
        -8.68036032e-01, -5.11364281e-01,  7.64371395e-01,
        -7.78510809e-01,  1.61245868e-01,  9.96432662e-01,
        -8.50842774e-01,  2.55859792e-01,  6.82226777e-01,
         4.60891932e-01,  5.45784414e-01,  3.72807086e-02,
         9.34222043e-01, -7.69549131e-01, -6.39944255e-01,
         3.43927830e-01,  7.79626966e-01, -4.83719647e-01,
        -8.17048475e-02, -6.28001153e-01,  7.36500263e-01,
        -9.27567661e-01,  4.30933505e-01,  5.27823985e-01,
        -5.12674272e-01, -2.93554142e-02, -8.74769613e-02,
         9.82195139e-01,  9.56808150e-01,  9.85575795e-01,
         7.03560174e-01,  7.30676949e-01, -5.22088744e-02,
        -6.91970468e-01, -5.84005117e-01,  5.29538095e-01,
        -9.48682129e-01,  2.36621350e-01, -8.61436486e-01,
         8.59434456e-02,  8.67155015e-01, -9.75041807e-01,
        -6.07386231e-01, -7.71296620e-01,  1.10219000e-03,
      

### Convert Tensor array into numpy array

In [29]:
# convert tf to ndarray
new_emb = bert_results.numpy()

In [31]:
print(type(new_emb))
print(type(new_emb[0]))
print(type(new_emb[0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


# Install Chroma db

In [ ]:
!pip install chromadb

In [2]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="tempdb")

In [32]:
collection.add(
    embeddings=new_emb,
    documents=my_text,
    ids=[str(n) for n in range(len(my_text))]
)

In [52]:
print(collection.get(include=['embeddings', 'documents', 'metadatas']))

{'ids': ['0', '1', '2', '3', '4'], 'embeddings': [[-0.21247504651546478, 0.6905425190925598, -0.1173485815525055, -0.8680360317230225, -0.5113642811775208, 0.764371395111084, -0.7785108089447021, 0.16124586760997772, 0.9964326620101929, -0.8508427739143372, 0.2558597922325134, 0.6822267770767212, 0.4608919322490692, 0.5457844138145447, 0.03728070855140686, 0.9342220425605774, -0.7695491313934326, -0.6399442553520203, 0.3439278304576874, 0.7796269655227661, -0.4837196469306946, -0.08170484751462936, -0.6280011534690857, 0.7365002632141113, -0.9275676608085632, 0.43093350529670715, 0.5278239846229553, -0.5126742720603943, -0.02935541421175003, -0.08747696131467819, 0.9821951389312744, 0.9568081498146057, 0.985575795173645, 0.7035601735115051, 0.7306769490242004, -0.05220887437462807, -0.6919704675674438, -0.5840051174163818, 0.529538094997406, -0.9486821293830872, 0.23662135004997253, -0.8614364862442017, 0.08594344556331635, 0.8671550154685974, -0.9750418066978455, -0.6073862314224243, 

In [46]:
# convert query into embeddings
my_query_text = ["When did rohit sharma born?"]
query_sentences = tf.constant(my_query_text)
bert_query_results = embedding_model(query_sentences)
bert_query_results

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-0.96658385,  0.9321441 , -0.0759258 , -0.9865395 , -0.99628484,
        -0.30991414, -0.324935  ,  0.10139401,  0.9722349 , -0.911723  ,
        -0.78567696,  0.8138744 ,  0.21320502,  0.9936597 ,  0.03152294,
         0.6486901 , -0.9977474 ,  0.8451004 , -0.76919997,  0.9810448 ,
        -0.9633885 , -0.66889834, -0.20063609,  0.9775524 ,  0.46994236,
         0.76870084, -0.6685349 , -0.9669014 , -0.04835703,  0.55693233,
         0.9920608 ,  0.94421613,  0.8938583 ,  0.9736169 ,  0.99249494,
        -0.04734606, -0.96781075, -0.8460368 , -0.3053102 ,  0.07791122,
        -0.9955416 , -0.97184265,  0.14217067,  0.9987265 , -0.843662  ,
        -0.95144856, -0.00316563, -0.96015763,  0.9707736 ,  0.9987506 ,
        -0.83700955, -0.7769336 ,  0.1929552 ,  0.93548214,  0.10179101,
        -0.73378396, -0.01811321,  0.20085436, -0.9788663 ,  0.99197114,
        -0.98665327, -0.7899409 , -0.15292923, -0.07586733, -0.87817   ,
 

In [47]:
# convert query tensor array into numpy array
new_query_embeddings = bert_query_results.numpy()

In [55]:
# fetch data
results = collection.query(
    query_texts=my_query_text,
    n_results=2,
    query_embeddings=new_query_embeddings,
    include=["embeddings"]
)

results

ValueError: You must provide one of query_embeddings, query_texts, query_images, or query_uris.

In [36]:
bert_results.shape

TensorShape([5, 128])

In [40]:
type(new_emb)

numpy.ndarray

In [42]:
new_emb.shape

(5, 128)